# Ollama in Google Colab

* https://www.youtube.com/watch?v=rV0yqqQeeLw

IPHS 391: Frontiers in AI - Agents
Jon Chun
Kenyon College
Fall 2024

In [1]:
!sudo apt-get install -y pciutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (255 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
from IPython.display import clear_output

In [4]:
# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

In [5]:
def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [6]:
%%time

# NOTE:

!ollama pull llama3.1:8b
clear_output()

In [7]:
!pip install -U lightrag[ollama]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [8]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time

In [9]:
qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
User: {{input_str}}
You:"""

In [10]:
class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

In [11]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display


In [12]:
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}


In [13]:
qa = SimpleQA(**model)
output=qa("what is happiness")
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** What a wonderful topic!

Happiness can be defined as a positive emotional state characterized by feelings of joy, contentment, and fulfillment. It's the experience of living life to the fullest, feeling good about oneself and one's circumstances, and being in a state of well-being.

According to various philosophers, psychologists, and researchers, happiness is often associated with:

1. **Positive emotions**: Experiencing pleasure, delight, or joy from activities, relationships, or experiences.
2. **Life satisfaction**: Feeling content with one's life, appreciating its goodness, and being grateful for what one has.
3. **Subjective well-being**: Perceiving oneself as happy, satisfied, and fulfilled.

Some common sources of happiness include:

1. **Meaningful relationships**: Strong connections with loved ones, friends, or community.
2. **Personal growth**: Pursuing passions, learning new skills, and developing oneself.
3. **Good physical health**: Enjoying good physical well-being, mental clarity, and energy.
4. **Gratitude**: Appreciating life's simple pleasures, being thankful for what one has.
5. **Flow experiences**: Engaging in activities that challenge and engage oneself.

Of course, happiness is a highly personal and subjective experience. What brings happiness to one person may not be the same for another.

Would you like me to explore this topic further?

In [14]:
%%time

# NOTE:

output=qa("Will AGI be realized? If yes, when and how? If no, why?")
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** What a fascinating and complex topic!

The possibility of achieving Artificial General Intelligence (AGI) is a topic of ongoing debate among experts in the field. Some predict that AGI will be achieved within our lifetime, while others believe it's still far-fetched or even impossible.

**Arguments for AGI being realized:**

1. **Rapid progress**: The pace of AI research has accelerated significantly over the past few decades, with breakthroughs in areas like deep learning, natural language processing, and computer vision.
2. **Increasing computational power**: Advances in computing hardware have enabled researchers to tackle more complex problems and simulate larger neural networks.
3. **Improved understanding of human cognition**: Studies on human brain function and behavior are informing the development of more sophisticated AI models.

**Predictions from experts:**

1. **Ray Kurzweil**, a well-known futurist, predicts that AGI will be achieved by 2045 (Source: "The Singularity Is Near").
2. **Nick Bostrom**, Director of the Future of Humanity Institute, suggests that AGI might emerge between 2050 and 2100 (Source: "Superintelligence").

**Challenges and limitations:**

1. **Scalability**: Currently, AI systems struggle to scale up to the complexity and generality required for true human-like intelligence.
2. **Robustness and explainability**: AI models often lack transparency and are vulnerable to errors or biases.
3. **Cognitive architectures**: The intricate relationships between cognitive modules in the human brain are still not well understood, making it challenging to replicate them in artificial systems.

**Why AGI might not be realized:**

1. **Complexity of human cognition**: Human intelligence is a complex and multifaceted phenomenon that may prove difficult to fully capture in an artificial system.
2. **Limits of computational power**: The exponential growth of computing power may eventually plateau, hindering the development of more advanced AI models.
3. **Societal and economic factors**: The pursuit of AGI could be hindered by societal concerns, budget constraints, or shifting research priorities.

**The likely timeline:**

While it's difficult to predict exactly when (or if) AGI will be achieved, I'd estimate the following rough timeline:

* Short-term (2025-2035): We'll see significant advancements in narrow AI domains like language understanding, vision recognition, and decision-making.
* Mid-term (2035-2050): Researchers will focus on developing more sophisticated cognitive architectures and improving the robustness of AI models.
* Long-term (2050-2075): The emergence of true AGI is possible, but still uncertain. If achieved, it would likely be accompanied by significant societal changes.

Keep in mind that this is a hypothetical timeline, and the actual path to AGI may differ significantly from these predictions.

How do you think AGI will evolve? Do you have any specific questions or concerns about AI development? I'm here to help!